In [1]:
import numpy as np
import pandas as pd
import json
import os
from sklearn.model_selection import KFold

In [2]:
# ฟังก์ชันสำหรับอ่าน JSON หลายบรรทัด
def read_multiline_json(file_path):
    """
    อ่านไฟล์ JSON ที่มีหลายบรรทัดและแปลงเป็น list ของ dictionary
    """
    try:
        with open(file_path, 'r') as file:
            return [json.loads(line) for line in file]
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading JSON file at {file_path}: {e}")
        return []

# ฟังก์ชันสำหรับกรอง matrix (ลบทั้ง row และ column)
def filter_matrix_full(matrix, indices):
    """
    กรองข้อมูลใน matrix โดย:
    - validation_test_matrix: เปลี่ยนค่าแถว indices เป็น 0 แล้วเลือกเฉพาะคอลัมน์ของ indices
    - validation_train_matrix: ลบทั้งแถวและคอลัมน์ของ indices

    Parameters:
        matrix (np.ndarray): Matrix ที่ต้องการกรอง
        indices (list): Index ของแถวที่จะลบหรือเปลี่ยนค่า

    Returns:
        tuple: (validation_train_matrix, validation_test_matrix)
    """
    try:
        # สร้างสำเนา matrix เพื่อไม่แก้ไขต้นฉบับ
        matrix_copy = matrix.copy()
        
        # เปลี่ยนค่าแถวที่มี indices เป็น 0
        matrix_copy[indices, :] = 0
        
        # validation_test_matrix: เลือกเฉพาะคอลัมน์ของ indices
        validation_test_matrix = matrix_copy[:, indices]

        # validation_train_matrix: ลบทั้งแถวและคอลัมน์ของ indices
        validation_train_matrix = np.delete(matrix, indices, axis=0)
        validation_train_matrix = np.delete(validation_train_matrix, indices, axis=1)

        return validation_train_matrix, validation_test_matrix
    except IndexError as e:
        print(f"Error filtering matrix: {e}")
        return matrix, None

# ฟังก์ชันสำหรับแบ่งข้อมูล 4-fold และกรอง matrix
def create_folds_and_filter_matrix(data, matrix, n_splits=4):
    """
    แบ่งข้อมูลเป็น 4-fold และกรอง matrix โดยแยก validation_train_matrix และ validation_test_matrix

    Parameters:
        data (pd.DataFrame): ข้อมูลต้นฉบับ
        matrix (np.ndarray): Matrix ที่ต้องการกรอง
        n_splits (int): จำนวน fold สำหรับ KFold

    Returns:
        list: รายการของ tuple (train_data, test_data, validation_train_matrix, validation_test_matrix)
    """
    kf = KFold(n_splits=n_splits, shuffle=False)
    folds = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index]
        test_data = data.iloc[test_index]

        # กรอง matrix โดยแยก validation_train_matrix และ validation_test_matrix
        validation_train_matrix, validation_test_matrix = filter_matrix_full(matrix, test_index)

        # เก็บข้อมูล train, test, validation_train_matrix และ validation_test_matrix
        folds.append((train_data, test_data, validation_train_matrix, validation_test_matrix))

    return folds

# ฟังก์ชันสำหรับบันทึกข้อมูลแต่ละ fold
def save_folds_as_json(folds, output_dir, prefix):
    """
    บันทึกข้อมูล train, test, validation_train_matrix และ validation_test_matrix ในแต่ละ fold เป็น JSON

    Parameters:
        folds (list): รายการของ fold (train_data, test_data, validation_train_matrix, validation_test_matrix)
        output_dir (str): โฟลเดอร์ที่ใช้บันทึกข้อมูล
        prefix (str): คำขึ้นต้นสำหรับชื่อไฟล์
    """
    os.makedirs(output_dir, exist_ok=True)

    for i, (train_data, test_data, validation_train_matrix, validation_test_matrix) in enumerate(folds):
        fold_dir = os.path.join(output_dir, f"fold_{i+1}")
        os.makedirs(fold_dir, exist_ok=True)

        # บันทึก train, test, validation_train_matrix และ validation_test_matrix เป็น JSON
        train_data.to_json(os.path.join(fold_dir, f"{prefix}_validation_train.json"), orient="records", lines=True)
        test_data.to_json(os.path.join(fold_dir, f"{prefix}_validation_test.json"), orient="records", lines=True)
        with open(os.path.join(fold_dir, f"{prefix}_validation_train_matrix.json"), "w") as f:
            json.dump(validation_train_matrix.tolist(), f)
        with open(os.path.join(fold_dir, f"{prefix}_validation_test_matrix.json"), "w") as f:
            json.dump(validation_test_matrix.tolist(), f)

In [ ]:
# Paths ของไฟล์
malware_matrix_path = r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\02.EDIT_DISTANCE_MATRIX\malware_distance_matrix_1000.npy'
benign_matrix_path = r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\02.EDIT_DISTANCE_MATRIX\benign_distance_matrix.npy'

benign_train_path = r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\benign_train.json'
malware_train_path = r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_train_1000.json'

# โหลดไฟล์ .npy
try:
    malware_matrix = np.load(malware_matrix_path)
    benign_matrix = np.load(benign_matrix_path)
except FileNotFoundError as e:
    print(f"Error loading matrix file: {e}")
    malware_matrix = benign_matrix = None

# อ่านไฟล์ JSON
benign_train_data = read_multiline_json(benign_train_path)
malware_train_data = read_multiline_json(malware_train_path)

# แปลงข้อมูล JSON เป็น DataFrame
benign_df = pd.DataFrame(benign_train_data)
malware_df = pd.DataFrame(malware_train_data)

# ตรวจสอบว่า matrix และ data ไม่เป็น None ก่อนดำเนินการ
if benign_matrix is not None and not benign_df.empty:
    benign_folds = create_folds_and_filter_matrix(benign_df, benign_matrix)
    save_folds_as_json(benign_folds, r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.CROSS_VALIDATION_4_FOLD\BENIGN_FOLDS', 'benign')
else:
    benign_folds = []
    print("Skipping benign data processing due to missing or empty data.")

if malware_matrix is not None and not malware_df.empty:
    malware_folds = create_folds_and_filter_matrix(malware_df, malware_matrix)
    save_folds_as_json(malware_folds, r'C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.CROSS_VALIDATION_4_FOLD\MALWARE_FOLDS', 'malware')
else:
    malware_folds = []
    print("Skipping malware data processing due to missing or empty data.")

# สรุปผลลัพธ์
print(f"Processed {len(benign_folds)} folds for benign data.")
print(f"Processed {len(malware_folds)} folds for malware data.")